In [1]:
import os
from sqlalchemy import create_engine, Column, Integer, String, Date, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from datetime import datetime
from langchain import LLMChain, PromptTemplate
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = "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"

In [2]:
Base = declarative_base()

class Expense(Base):
    __tablename__ = 'expenses'
    id = Column(Integer, primary_key=True)
    date = Column(Date)
    category = Column(String)
    amount = Column(Integer)

# Create a database engine
engine = create_engine('sqlite:///expense.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

llm = ChatGroq(model="llama3-8b-8192", temperature=0)

/tmp/ipykernel_7297/679481064.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
prompt_template = """
You are an assistant that extracts expense details from the user's input. 
The input is: {user_input}
Extract the amount, category, and current date based on your knowledge.

For Example: if the user gives the input like "i have spend 100 for food today"
    Then you have to extract the data like Amount: 100 and the category: "Food" form the user input.

**always double check if the data's are extracted properly**

your response always need to like **example: Amount: 100, Category: food.**
only the amount and the category and always use ** before and after the Amount and Category text
** if can't able to find Category then return Category: unknown ** 

"""


In [52]:
prompt_template = """

    You are an assistant that extracts expense details from the user's input. 
The input is: {user_input}
Extract the amount, category, commend and current date based on your knowledge.

For Example: 
1) if the user gives the input like "i have spend 100 for food today"
Then you have to extract the data like Amount: 100 and the category: "Food" and
the commend is Unknown because the user not mension the reason in the user input.

2) if the user give the input like "i have spend 100 for car fuel today" then you have to extract
the data like Amount: 100, category: fuel and the commend: car. as the user mension that he spend 100 
for the car fuel.

**always double check if the data's are extracted properly**

your response always need to like **example: Amount: 100, Category: food. commend: car**
only the amount and the category and always use ** before and after the Amount and Category text
** if can't able to find Category then return Category: unknown ** 


"""

In [13]:

def extract_expense(user_input):
    prompt = PromptTemplate(input_variables=["user_input"], template=prompt_template)
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    
    response = llm_chain.run(user_input)
    

    # print(response)
    # return response
    # Parsing the output (You may adjust parsing based on Llama's output)
    amount = int(response.split("**")[1].split(",")[0].split(":")[1].strip())
    category = response.split("**")[1].split(",")[1].split(":")[1].strip()
    date = datetime.today().date()
    
    # If the category is unclear, the agent will ask the user for clarification.
    if category.lower() == "unknown":
        category = input("Which category does this spending come under? ")
    
    # Save the extracted data to the SQL database
    new_expense = Expense(date=date, category=category, amount=amount)
    session.add(new_expense)
    session.commit()
    
    return f"Expense added: {amount} spent on {category}."

In [11]:
def retrieve_expense(date=None, week=None, category=None):
    query = session.query(Expense)
    
    if date:
        query = query.filter(Expense.date == date)
    if week:
        start_week = datetime.today() - timedelta(days=datetime.today().weekday())
        end_week = start_week + timedelta(days=6)
        query = query.filter(Expense.date.between(start_week, end_week))
    if category:
        query = query.filter(Expense.category == category)
    
    total_spent = sum(expense.amount for expense in query.all())
    return total_spent
    # return query.all()


In [6]:
user_input = "Today I spent 100 for food"
extract_expense(user_input)

/tmp/ipykernel_7297/2728415002.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)
/tmp/ipykernel_7297/2728415002.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = llm_chain.run(user_input)


I'm ready to extract the expense details.

For the input: "Today I spent 100 for food"

I extracted the following details:

**Amount: 100, Category: food**

Please let me know if these details are correct or not!


'I\'m ready to extract the expense details.\n\nFor the input: "Today I spent 100 for food"\n\nI extracted the following details:\n\n**Amount: 100, Category: food**\n\nPlease let me know if these details are correct or not!'

In [8]:
print(retrieve_expense())

400


In [15]:
query = session.query(Expense)
for i in query.all():
    print(i.id, i.amount, i.date, i.category)

1 150 2024-09-19 Groceries
2 500 2024-09-19 Rent
3 80 2024-09-19 Car Fuel
4 12000 2024-09-19 Phone
5 200 2024-09-19 Restaurant
6 300 2024-09-19 Internet
7 50 2024-09-19 Public Transport
8 400 2024-09-19 Health Insurance
9 1000 2024-09-19 Clothing
10 150 2024-09-19 Subscriptions


In [8]:
test_inputs = [
    "Today I spent 150 on groceries.",
    "I paid 500 for rent this week.",
    "Yesterday I spent 80 on car fuel.",
    "I bought a new phone for 12000.",
    "Spent 200 on a restaurant with friends last night.",
    "Paid 300 for my internet bill today.",
    "I spent 50 on public transport this morning.",
    "I had to pay 400 for health insurance.",
    "I spent 1000 on clothing during the weekend.",
    "Spent 150 on subscriptions last month."
]


In [14]:
for i in test_inputs:
    extract_expense(i)

In [44]:
extract_expense(test_inputs[0]).split("**")[1].split(",")[0].split(":")[1].strip()

'150'